## Read Data File which is delivered by Source Team 

In [179]:
import pandas as pd

In [180]:
data= pd.read_csv('demo.csv', sep='|', index_col=0)

In [181]:
data.drop('H', axis='columns', inplace=True)

In [182]:
data.reset_index(drop=True, inplace=True)

In [183]:
data.head()

,Customer_Name,Customer_Id,Open_Date,Last_Consulted_Date,Vaccination_Id,Dr_Name,State,Country,DOB,Is_Active
0,John,123456,20101012,20121013,MVD,Paul,NSW,AU,6031987,A
1,Soham,123457,20101012,20121013,MVD,Paul,WB,IND,3031998,A


## Restructuring the Table Schema 

In [184]:
data= data.rename({'Open_Date':'Customer_Open_Date',
                   'Vaccination_Id':'Vaccination_Type',
                   'Dr_Name':'Doctor_Consulted',
                   'DOB':'Date_Of_Birth',
                   'Is_Active':'Active_Customer'}, axis='columns')

In [185]:
data.head()

,Customer_Name,Customer_Id,Customer_Open_Date,Last_Consulted_Date,Vaccination_Type,Doctor_Consulted,State,Country,Date_Of_Birth,Active_Customer
0,John,123456,20101012,20121013,MVD,Paul,NSW,AU,6031987,A
1,Soham,123457,20101012,20121013,MVD,Paul,WB,IND,3031998,A


In [186]:
data['Post_Code']= pd.Series(dtype= object)

In [187]:
data.columns

Index(['Customer_Name', 'Customer_Id', 'Customer_Open_Date',
       'Last_Consulted_Date', 'Vaccination_Type', 'Doctor_Consulted', 'State',
       'Country', 'Date_Of_Birth', 'Active_Customer', 'Post_Code'],
      dtype='object')

In [188]:
data=data[['Customer_Name', 'Customer_Id', 'Customer_Open_Date',
       'Last_Consulted_Date', 'Vaccination_Type', 'Doctor_Consulted', 'State',
       'Country','Post_Code', 'Date_Of_Birth', 'Active_Customer']]

In [189]:
data.head()

,Customer_Name,Customer_Id,Customer_Open_Date,Last_Consulted_Date,Vaccination_Type,Doctor_Consulted,State,Country,Post_Code,Date_Of_Birth,Active_Customer
0,John,123456,20101012,20121013,MVD,Paul,NSW,AU,NaN,6031987,A
1,Soham,123457,20101012,20121013,MVD,Paul,WB,IND,NaN,3031998,A


# Load the data in Staging File

In [205]:
from sqlalchemy import create_engine

In [206]:
engine = create_engine('sqlite:///demo_data.db', echo= True)

In [207]:
sqlite_connection=engine.connect()

2021-06-05 00:13:02,008 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-06-05 00:13:02,010 INFO sqlalchemy.engine.base.Engine ()
2021-06-05 00:13:02,012 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-06-05 00:13:02,014 INFO sqlalchemy.engine.base.Engine ()


In [208]:
sqlite_table='Customer'

In [209]:
data.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-06-05 00:13:04,106 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Customer")
2021-06-05 00:13:04,108 INFO sqlalchemy.engine.base.Engine ()


ValueError: Table 'Customer' already exists.

In [210]:
sqlite_connection.close()

# Creating table based on country 

In [242]:
import sqlite3

In [243]:
con= sqlite3.connect('demo_data.db')

In [244]:
cursor=con.cursor()

In [234]:
sql1='''
 CREATE TABLE IF NOT EXISTS TABLE_INDIA(
  Customer_Name VARCHAR(255) PRIMARY KEY,
  Customer_Id VARCHAR(18) NOT NULL,
  Customer_Open_Date DATE NOT NULL,
  Last_Consulted_Date DATE,
  Vaccination_Type CHAR(5), 
  Doctor_Consulted CHAR(255), 
  State CHAR(5),
  Country CHAR(5) ,
  Post_Code INTEGER,
  Date_Of_Birth DATE,
  Active_Customer CHAR(1)
  )
  '''

In [235]:
cursor.execute(sql1)

##  To check the record in Customer table 

In [245]:
sql2='''
     select Customer_Name, Customer_Id, Customer_Open_Date,
       Last_Consulted_Date, Vaccination_Type, Doctor_Consulted, State,
       Country,Post_Code, Date_Of_Birth, Active_Customer
       from Customer where Country='IND'
     '''

In [246]:
cursor.execute(sql2)

In [247]:
recs=cursor.fetchall()

In [248]:
for record in recs:
    x=record
    print(x)

('Soham', 123457, 20101012, 20121013, 'MVD', 'Paul', 'WB', 'IND', None, 3031998, 'A')


## Insert data in Table_India 

In [249]:
sql3='''
INSERT INTO TABLE_INDIA
select Customer_Name, Customer_Id, Customer_Open_Date,
       Last_Consulted_Date, Vaccination_Type, Doctor_Consulted, State,
       Country,Post_Code, Date_Of_Birth, Active_Customer
       from Customer where Country='IND'
       '''

In [250]:
cursor.execute(sql3)

OperationalError: database is locked

In [ ]:
con.commit()

## To check record in Table_India 

In [251]:
sql='''
select * from TABLE_INDIA
'''

In [252]:
cursor.execute(sql)

In [253]:
r=cursor.fetchall()

In [254]:
for i in r:
    print(i)

In [255]:
con.close()